# Применение статистических методов

Этот семинар призван помочь освоиться на практике со статистическими методами, которые изучались на лекции. [Ссылка](https://colab.research.google.com/drive/12JisoRiIP2RdVsZ8QvPC2MUHHeQGHx4U?usp=sharing) на тетрадку.

Вам будет дан ряд экспериментов — часть по реальным данным, часть по сгенерированным — и их результаты будет нужно необходимо статистически обработать.

Применяя каждый критерий, проходите по всем стадиям статистического вывода:

- сформулируйте нулевую и альтернативную гипотезы
- выберите уровень значимости (p<0.05, p<0.01, p<0.001)
- выберите тест на основании вида ваших данных
- произведите вычисления
- сделайте вывод о значимости различий
- сделайте итоговый вывод из эксперимента

Критерии, из которых следует выбирать в заданиях:

- t-критерий
  - одновыборочный
  - двухвыборочный
- U-критерий Манна-Уитни
- критерий $\chi^2$
- критерий Фишера

см. таблицу на предпоследнем слайде лекции, где обсуждается выбор критерия; ниже не встречается случаев, для которых нужен критерий Вилкоксона или биномиальный тест

Дополнительно, в конкретном задании вам может быть нужно изменить форму данных: преобразовать вид таблички или скопировать числа из нужного места.

## Тесты из Питона

Естественно, все перечисленные тесты можно запустить из Питона

- ttest() из модуля researchpy (есть все варианты [теста](https://researchpy.readthedocs.io/en/latest/ttest_documentation.html))
- chi2_contingency() модуля scipy.stats
- mannwhitneyu() модуля scipy.stats

и т. д.

В целом они применяются несложно — код ниже. Обычно самое сложное — понять, в каком формате подавать им данные

In [18]:
%%capture
!pip install scipy

In [31]:
import pandas as pd
from scipy import stats

In [3]:
#Берём датасет с двумя числовыми переменными (рост в двух популяциях из презентации),
#чтобы использовать t-критерий
samples = pd.read_csv('https://raw.githubusercontent.com/alekseyst/text_analysis_2024/main/Practical_8/sample_ttest.csv')
samples.head(10)

,sample_one,sample_two
0,174,182
1,171,171
2,195,162
3,162,175
4,179,142
5,178,166
6,181,161
7,173,169
8,169,158
9,191,182


In [35]:
#Запускаем t-критерий — получаем p-value
stats.ttest_ind(samples['sample_one'], samples['sample_two'], equal_var=False)

TtestResult(statistic=np.float64(2.842621098813265), pvalue=np.float64(0.00706167648655467), df=np.float64(39.30510720355258))

## Источник данных

Данные для вычисления статистических тестов можно подготовить внутри Питона: библиотеки для вычисления тестов хорошо совместимы с pandas.

In [5]:
#Скачиваем датасеты и добавляем столбец, который говорит, из какого датасета слово
alice = pd.read_csv('https://raw.githubusercontent.com/alekseyst/text_analysis_2025/main/Practical_4/alice_corpus_words.tsv', sep='\t')
alice['source'] = 'alice'

panorama = pd.read_csv('https://raw.githubusercontent.com/alekseyst/text_analysis_2025/main/Practical_4/panorama_corpus_words.tsv', sep='\t')
panorama['source'] = 'panorama'

In [6]:
#Соединяем две таблички
joint_corpus = pd.concat([alice, panorama])
joint_corpus

,index,word,lemma,pos,feats,synt,source,text_index,head
0,0,Льюис,Льюис,PROPN,Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing,root,alice,NaN,NaN
1,1,Кэрролл,Кэрролл,PROPN,Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing,flat:name,alice,NaN,NaN
2,2,Приключения,приключение,NOUN,Animacy=Inan|Case=Nom|Gender=Neut|Number=Plur,root,alice,NaN,NaN
3,3,Алисы,Алиса,PROPN,Animacy=Anim|Case=Gen|Gender=Fem|Number=Sing,nmod,alice,NaN,NaN
4,4,в,в,ADP,NaN,case,alice,NaN,NaN
...,...,...,...,...,...,...,...,...,...
119543,154,ставить,ставить,VERB,Aspect=Imp|VerbForm=Inf|Voice=Act,xcomp,panorama,624.0,14.0
119544,155,вопрос,вопрос,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obj,panorama,624.0,15.0
119545,156,подобным,подобный,ADJ,Case=Ins|Degree=Pos|Gender=Masc|Number=Sing,amod,panorama,624.0,18.0
119546,157,образом,образ,NOUN,Animacy=Inan|Case=Ins|Gender=Masc|Number=Sing,obl,panorama,624.0,15.0


In [17]:
#Фильтруем нужные слова, аггрегируем данные, делаем табличку

joint_corpus_old_new = joint_corpus[joint_corpus['lemma'].isin(['старый', 'новый'])]
contingency_table = pd.crosstab(joint_corpus_old_new['lemma'], joint_corpus_old_new['source'])
contingency_table

source,alice,panorama
lemma,,
новый,9,295
старый,11,25


In [37]:
#Применяем хи-квадрат

stats.chi2_contingency(contingency_table)

Chi2ContingencyResult(statistic=np.float64(39.42871093750001), pvalue=np.float64(3.4025714345892356e-10), dof=1, expected_freq=array([[ 17.88235294, 286.11764706],
       [  2.11764706,  33.88235294]]))

При этом совсем не обязательно использовать языки программирования, чтобы посчитать тесты: можно просто зайти на сайт, выбрать тест, ввести данные и получить результаты.

Опасность состоит в том, что некоторые сайты считают с ошибками. Но вот, например, есть [такой](http://vassarstats.net/). Хи-квадрат там находится во вкладке Clinical Research Calculators и, возможно, устроен сложновато. Если не нравится — можете выбрать понравившийся вам сайт из интернета. Вот [ещё](https://www.socscistatistics.com/) сайт, там дизайн менее затейлевый и есть ограничение по количеству вводимых данных, но пользоваться попроще.

# Задание 1

Посчитайте наконец, значимо ли Толстой пишет длиннее, чем Чехов!

https://raw.githubusercontent.com/alekseyst/text_analysis_2024/main/Practical_8/author.csv

# Задание 2

Известно, что слово *друг* в милионном простом корпусе встретилось 193 раз, а в газетном — 447 раз; слово *враг* — 36 и 46 раз соответственно. Есть ли различия в употреблении на уровне значимости 0.01? 0.05? Как интерпретировать эти различия?




# Задание 3

Вы проводили социологический среди программистов и лингвистов, прося оценить от 1 до 5 удовлетворённость зарплатой, и получили следующие ответы:

Программисты: 5, 4, 5, 5, 5, 3, 3, 3, 5, 3, 3, 3, 5, 3, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 4, 4, 5, 5, 5

Лингвисты: 1, 5, 3, 3, 5, 5, 2, 5, 4, 4, 2, 2, 3, 2, 3, 2, 3, 3, 2, 3, 5, 3, 5, 5, 5

Правда ли, что ответы этих двух групп различаются? (Обратите внимание на то, как выглядят данные.)



# Задание 4

Известно, что четырёхглазые тушканчики с далёкой планеты по данным переписи в среднем в быстрой речи произносят 2.86 слога в секунду; стандартное отклонение 0.34.

Тушканчики одной из стран, в соответствии с распространёнными на планете стереотипами, говорят гораздо медленнее, чем остальные (про это есть много анекдотов).

45 разных тушканчиков этой страны были записаны и произнесли в секунду следующие количества слогов. Правда ли они говорят значимо медленнее, или это только стереотип?

3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3

# Задание 5

Вы ведёте переписку с двумя людьми: один — ваш хороший знакомый Фёдор Михайлович, другой — тоже хороший знакомый Иван Петрович. Оба из них используют в письмах очень много обращений. В какой-то момент они пошли на литературные курсы, после которых Иван Петрович, по его утверждению, решил, что такое количество обращений — дурной вкус, А Федор Михайлович ничего не прокомментировал. Вы посчитали количество обращений до похода и после в письмах обоих друзей, чтобы определить, правда ли один из них всё тот же, а другой изменился.

Количество обращений в письме Ф. М. после похода на курсы — 12, после — 11, у И. П. 14, после — 2.